## Step 1. 데이터 합치기 / Aggregation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
# salary_1 이름으로 salary_1.csv 불러오기 (데이터 위치: 'data/salary_1.csv')
salary_1 = pd.read_csv('./data/salary_1.csv')

In [4]:
# salary_2 이름으로 salary_2.csv 불러오기 (데이터 위치: 'data/salary_2.csv')
salary_2 = pd.read_csv('./data/salary_2.csv')

In [5]:
# salary_1 과 salary_2를 위/아래로 붙이고, salary_df 이름으로 저장하기
salary_df = pd.concat([salary_1, salary_2])

In [6]:
# salary_df의 결측치 비율 확인하기
salary_df.isna().mean()

Age                    0.000598
Gender                 0.000000
Education Level        0.000000
Job Title              0.000000
Years of Experience    0.000000
Salary                 0.000000
Country                0.000000
Race                   0.000000
Senior                 0.000000
dtype: float64

In [7]:
# 결측치 행 제거하기
salary_df = salary_df.dropna()

In [8]:
# Gender별 Salary의 평균(mean)을 구해 gender_salary 로 저장하기 
gender_salary = salary_df.groupby('Gender')['Salary'].mean()

In [9]:
# gender_salary의 인덱스(Gender)를 컬럼으로 전환하여 저장하기
gender_salary = gender_salary.reset_index()

In [10]:
# Gender를 기준으로 하여, salary_df에 gender_salary를 붙여서(left join) salary_df로 저장하기
salary_df = salary_df.merge(gender_salary, on = 'Gender', how = 'left')

In [11]:
# 컬럼이름 변경: "Salary_x"를 "Salary"로, "Salary_y"를 "Gender_salary"로 변경하여 저장
salary_df = salary_df.rename({'Salary_x': 'Salary', 'Salary_y': 'Gender_salary'}, axis = 1)

In [12]:
# 다음 기준으로 Pivot Table 만들기 -> 행: Country, 열: Gender, 값: Years of Experience
pd.pivot_table(salary_df, index='Country', columns='Gender', values='Years of Experience')

Gender,Female,Male
Country,,
Australia,7.397227,8.493065
Canada,7.212199,8.878214
China,7.985618,8.444519
UK,7.575658,8.781466
USA,7.036526,8.475643


## Step 2. 데이터 스케일 변환하기

In [13]:
# Salary 변수에 로그를 취하여 'Salary_log'로 저장하기
salary_df['Salary_log'] = np.log(salary_df['Salary'])

In [16]:
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6680 entries, 0 to 6679
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6680 non-null   float64
 1   Gender               6680 non-null   object 
 2   Education Level      6680 non-null   int64  
 3   Job Title            6680 non-null   object 
 4   Years of Experience  6680 non-null   float64
 5   Salary               6680 non-null   int64  
 6   Country              6680 non-null   object 
 7   Race                 6680 non-null   object 
 8   Senior               6680 non-null   int64  
 9   Gender_salary        6680 non-null   float64
 10  Salary_log           6680 non-null   float64
dtypes: float64(4), int64(3), object(4)
memory usage: 574.2+ KB


In [17]:
# 변수 중 Data Type이 object인 변수들 제거하고 저장하기
salary_df.drop(['Gender', 'Job Title', 'Country', 'Race'], axis = 1, inplace=True)

In [18]:
salary_df.head()

,Age,Education Level,Years of Experience,Salary,Senior,Gender_salary,Salary_log
0,32.0,1,5.0,90000,0,121393.353134,11.407565
1,28.0,2,3.0,65000,0,107888.555814,11.082143
2,45.0,3,15.0,150000,1,121393.353134,11.918391
3,36.0,1,7.0,60000,0,107888.555814,11.002100
4,52.0,2,20.0,200000,0,121393.353134,12.206073


In [21]:
!pip install scikit-learn

   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   --- ------------------------------------ 1.0/10.7 MB 8.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.7 MB 4.0 MB/s eta 0:00:03
   --------------------------- ------------ 7.3/10.7 MB 13.6 MB/s eta 0:00:01
   ----------------------------- ---------- 7.9/10.7 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 10.7/10.7 MB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   --------- ------------------------------ 10.2/41.0 MB 48.7 MB/s eta 0:00:01
   -------------------- ------------------- 21.0/41.0 MB 49.5 MB/s eta 0:00:01
   ----------------------------- ---------- 30.4/41.0 MB 48.3 MB/s eta 0:00:01
   ------------------------------------- -- 38.5/41.0 MB 45.5 MB/s eta 0:00:01
   ---------------------------------------- 41.0/41.0 MB 41.3 MB/s eta 0:00:00

   ---------- ----------------------------- 1/4 [scipy]
   ---------- -----

In [22]:
# RobustScaler 패키지 불러오기
from sklearn.preprocessing import RobustScaler

In [23]:
# RobustScaler를 사용하기 위해 rs 이름으로 저장
rs = RobustScaler()

In [24]:
# rs로 salary_df의 정보를 학습시키기
rs.fit(salary_df)

,with_centering,True
,with_scaling,True
,quantile_range,"(25.0, ...)"
,copy,True
,unit_variance,False


In [25]:
# 학습된 rs로 salary_df를 변환하여 rs_df로 저장하기
rs_df = rs.transform(salary_df)

In [27]:
# rs_df를 Pandas DataFrame으로 변경하여 저장 (컬럼 이름도 기존 컬럼이름으로 채워넣기)
rs_df = pd.DataFrame(rs_df, columns = salary_df.columns)

In [28]:
# MinMaxScaler 패키지 불러오기
from sklearn.preprocessing import MinMaxScaler

In [31]:
# MinMaxScaler를 사용하기 위해 mm 이름으로 저장
mm = MinMaxScaler()

In [32]:
# 한 줄의 코드로 salary_df를 mm으로 학습하고 변형하여 mm_df 로 저장하기
mm_df=mm.fit_transform(salary_df)

In [33]:
# mm_df를 Pandas DataFrame으로 변경하여 저장 (컬럼 이름도 기존 컬럼이름으로 채워넣기)
mm_df = pd.DataFrame(mm_df, columns = salary_df.columns)

## Step 3. PCA

In [34]:
# PCA 패키지 불러오기
from sklearn.decomposition import PCA

In [35]:
# PCA를 사용하기 위해 pca 이름으로 저장: 2개의 주성분을 뽑을 수 있도록 설정
pca =  PCA(2)

In [36]:
# pca로 salary_df를 학습 및 변환하여 pca_df로 저장
pca_df = pca.fit_transform(salary_df)

In [37]:
# 추출된 두개의 주성분으로 기존 데이터 정보의 얼마만큼을 설명할 수 있는지 확인하는 코드 작성
(pca.explained_variance_ratio_).sum()

np.float64(0.9999999855512034)